In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import mediapy
import cv2
sys.path.append('../package/mujoco_helper/') # for 'mujoco_parser'
sys.path.append('../package/kinematics_helper/') # for 'transforms'
sys.path.append('../package/utility/') # for 'utils'
from mujoco_parser import *
from transforms import *
from utils import *
from ik import *
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:",MUJOCO_VERSION)

MuJoCo version: (3, 1, 6)


In [2]:
xml_path = '../asset/scene_car.xml'
env = MuJoCoParserClass(name='Car Scene',rel_xml_path=xml_path,verbose=False)

colors = plt.get_cmap('tab20').colors + plt.get_cmap('tab20b').colors[:env.model.ngeom]
for i in range(env.model.ngeom):
    geom_name = mujoco.mj_id2name(env.model, mujoco.mjtObj.mjOBJ_GEOM, i)
    if geom_name != 'floor': env.model.geom_rgba[i][:3] = colors[i]
    # if geom_name != 'floor': env.model.geom_rgba[i][:3] = [0.3,0.3,0.3]

In [3]:
env.reset()
env.init_viewer(
    azimuth           = 180,
    distance          = 10.5,
    elevation         = -8.8,
    lookat            = [-0.06,-0.71,1.26],
    transparent       = False,
    black_sky         = True,
    use_rgb_overlay   = True,
    loc_rgb_overlay   = 'top right',
    use_rgb_overlay_2 = True,
    loc_rgb_overlay_2 = 'top left',
)

p_start = np.array([2.5,-3.0,3.0])
p_end = np.array([2.5,1.8,3.0])
R_start = rpy2r_order(np.radians([0,45,90]))
tick = 0
max_tick = 100
point_cloud = np.zeros((0,3))

while env.is_viewer_alive() and tick < max_tick:
    env.step()
    env.data.cam('external').xpos = p_start + (p_end - p_start) * tick / max_tick
    env.data.cam('external').xmat = np.reshape(R_start,9)

    rgb_cam, depth_cam, pcd_cam, T_cam = env.get_fixed_cam_rgbd_pcd(
        cam_name='external',
        downscale_pcd = 0.5
    )
    _, seg_cam = env.get_fixed_cam_seg(cam_name='external')

    env.plot_T(T=T_cam,axis_len=0.1,axis_width=0.005,label="External")

    env.viewer.add_rgb_overlay(seg_cam)
    env.viewer.add_rgb_overlay_2(depth_to_gray_img(depth_cam))

    env.render()
    tick += 1

    pcd_cam = pcd_cam[pcd_cam[:,2] > 0.01]
    point_cloud = np.vstack([point_cloud,pcd_cam])

env.close_viewer()

2024-12-02 18:14:43.894 python[15921:1658116] +[IMKClient subclass]: chose IMKClient_Modern
2024-12-02 18:14:43.894 python[15921:1658116] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [4]:
import open3d as o3d

def custom_draw_geometry(pcd):
    vis = o3d.visualization.VisualizerWithKeyCallback()
    vis.create_window()
    vis.add_geometry(pcd)
    
    # Define callback function for ESC key
    def close_callback(vis):
        vis.destroy_window()
        return False
    
    # Register the callback function with key "Escape" (key value 256)
    vis.register_key_callback(256, close_callback)
    
    # Run the visualizer
    vis.run()
    vis.destroy_window()

# Create your point cloud
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud)

# Run the custom visualization
custom_draw_geometry(pcd)

objc[15921]: Class GLFWWindowDelegate is implemented in both /opt/homebrew/Cellar/glfw/3.3.9/lib/libglfw.3.3.dylib (0x171829858) and /Users/sy/miniforge3/envs/vision/lib/python3.9/site-packages/open3d/cpu/pybind.cpython-39-darwin.so (0x332e86da8). One of the two will be used. Which one is undefined.
objc[15921]: Class GLFWApplicationDelegate is implemented in both /opt/homebrew/Cellar/glfw/3.3.9/lib/libglfw.3.3.dylib (0x171829830) and /Users/sy/miniforge3/envs/vision/lib/python3.9/site-packages/open3d/cpu/pybind.cpython-39-darwin.so (0x332e86e20). One of the two will be used. Which one is undefined.
objc[15921]: Class GLFWContentView is implemented in both /opt/homebrew/Cellar/glfw/3.3.9/lib/libglfw.3.3.dylib (0x1718298a8) and /Users/sy/miniforge3/envs/vision/lib/python3.9/site-packages/open3d/cpu/pybind.cpython-39-darwin.so (0x332e86e48). One of the two will be used. Which one is undefined.
objc[15921]: Class GLFWWindow is implemented in both /opt/homebrew/Cellar/glfw/3.3.9/lib/libglf

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
